# 05: Perform LCIA

In this script, we perform LCIA on the whole selection of technologies.

In [4]:
%run common_definitions.py

In [5]:
import pandas as pd
import bw2calc as bc
import bw2data as bd

import time
import csv

In [6]:
bd.projects.set_current(BW_PROJECTNAME_BATTERIES)
output_fp = "../output/" + BW_PROJECTNAME

### Search for activities

In [7]:
def get_dbname(scen, year):
    matches = [db for db in bd.databases if "{}_{}".format(scen, year) in db]

    return matches[0]

In [9]:
search_str = "market for battery capacity, stationary"

activities = {}
for scen in SCENARIOS_BATTERIES:
    activities[scen] = {}
    for year in YEARS:
        dbname = get_dbname(scen, year)
        db = bd.Database(dbname)
        all_matches = [act for act in db if search_str in act["name"]]
        print("{} activities found in database {}".format(len(all_matches), dbname))
        activities[scen][year] = all_matches

2 activities found in database ei_cutoff_3.9_remind_SSP2-NPi_2020 2025-07-30
2 activities found in database ei_cutoff_3.9_remind_SSP2-NPi_2030 2025-07-30
2 activities found in database ei_cutoff_3.9_remind_SSP2-NPi_2040 2025-07-30
2 activities found in database ei_cutoff_3.9_remind_SSP2-NPi_2050 2025-07-30
2 activities found in database ei_cutoff_3.9_remind_SSP2-PkBudg650_2020 2025-07-30
2 activities found in database ei_cutoff_3.9_remind_SSP2-PkBudg650_2030 2025-07-30
2 activities found in database ei_cutoff_3.9_remind_SSP2-PkBudg650_2040 2025-07-30
2 activities found in database ei_cutoff_3.9_remind_SSP2-PkBudg650_2050 2025-07-30


### Get needed LCIA methods

In [10]:
df2 = pd.read_csv("../data/mfs_all_factors.csv").dropna(subset="LCIA method")

In [11]:
needed_methods = list(df2["LCIA method"].unique())
methods = [m for m in list(bd.methods) if ", ".join(list(m)) in needed_methods]

In [12]:
cc_methods = [
    ("IPCC 2021", "climate change: biogenic", "GWP 100a"),
    ("IPCC 2021", "climate change: land use", "GWP 100a"),
    ("IPCC 2021", "climate change: fossil", "GWP 100a")
]
methods = methods + cc_methods

In [13]:
print("{} methods needed, {} methods found".format(len(needed_methods)+len(cc_methods), len(methods)))

73 methods needed, 73 methods found


### Set up file for LCIA output

In [19]:
# open file for writing data
with open(output_fp + "/impacts_battery_storage.csv", "w", newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    row = ["ecoinvent name",
           "location", "unit", "amount", "product",
           "scenario", "year",
           ]
    row += [", ".join(list(m)) for m in methods]
    writer.writerow(row)

In [20]:
csvfile = open(output_fp + "/impacts_battery_storage.csv", "a", newline='')
writer = csv.writer(csvfile, delimiter=',')

for scen in SCENARIOS_BATTERIES:
    for year in YEARS:
        act_list = activities[scen][year]
        functional_units = [{act: 1} for act in act_list]

        new_setup = {"inv": functional_units, "ia": methods}
        setup_name = "battery_storage_{}_{}".format(scen, year)
        bd.calculation_setups[setup_name] = new_setup

        for act in act_list:
            lca = bc.LCA(demand={act: 1})
            lca.lci()
            scores = []
            for m in methods:
                lca.switch_method(m)
                lca.lcia()
                scores.append(lca.score)
            print("{} done.".format(str(act)))
            row = [act["name"],
                act["location"], act["unit"], str(1), act["reference product"],
                scen, str(year)] + [str(score) for score in scores]
            writer.writerow(row)         

csvfile.close()

'market for battery capacity, stationary (TC scenario)' (kilowatt hour, GLO, None) done.
'market for battery capacity, stationary (CONT scenario)' (kilowatt hour, GLO, None) done.
'market for battery capacity, stationary (TC scenario)' (kilowatt hour, GLO, None) done.
'market for battery capacity, stationary (CONT scenario)' (kilowatt hour, GLO, None) done.
'market for battery capacity, stationary (CONT scenario)' (kilowatt hour, GLO, None) done.
'market for battery capacity, stationary (TC scenario)' (kilowatt hour, GLO, None) done.
'market for battery capacity, stationary (CONT scenario)' (kilowatt hour, GLO, None) done.
'market for battery capacity, stationary (TC scenario)' (kilowatt hour, GLO, None) done.
'market for battery capacity, stationary (TC scenario)' (kilowatt hour, GLO, None) done.
'market for battery capacity, stationary (CONT scenario)' (kilowatt hour, GLO, None) done.
'market for battery capacity, stationary (TC scenario)' (kilowatt hour, GLO, None) done.
'market for